The purpose of this notebook is to download the assembled contigs, blast them against nt and nr, then return those contigs to AWS.

1. Set up an m5a.12xlarge instance, with keys and ports for jupyter from the `czbiohub-miniconda` AMI. Storage is at `/mnt/data`.

`aegea launch --iam-role S3fromEC2 --ami-tags Name=czbiohub-jupyter -t m5a.12xlarge  batson-blast`

Setup jupyter on the remote

`aegea ssh batson@batson-blast`
`tmux`
`jupyter notebook`

Port forwarding from laptop

`aegea ssh batson@batson-blast -NL localhost:8899:localhost:8888`

2. Download contigs

`mkdir /mnt/data/contigs`
`aws s3 sync s3://czbiohub-mosquito/contigs/ /mnt/data/contigs --exclude "*" --include "*.fasta" --dryrun`

3. Install requirements

conda install -c bioconda -c conda-forge blast

mkdir /mnt/data/blast
cd /mnt/data/blast
update_blastdb.pl --decompress nt nr taxdb

4. Run BLAST

Loop over all contigs, and run for each sample

`BLASTDB=/mnt/data/blast blastn -db nt -num_threads 48 -query /mnt/data/contigs/{SAMPLE}/contigs.fasta -outfmt 7 -out /mnt/data/contigs/{SAMPLE}/blastn_nt.m9 -evalue 1e-1`

`BLASTDB=/mnt/data/blast blastx -db nr -num_threads 48 -query /mnt/data/contigs/{SAMPLE}/contigs.fasta -outfmt 7 -out /mnt/data/contigs/{SAMPLE}/blastx_nr.m9 -evalue 1e-1`

5. Upload samples

`aws s3 sync /mnt/data/contigs/ s3://czbiohub-mosquito/contigs/ --exclude "*" --include "*.m9" --dryrun`

To download the contigs, we will sync to

`s3://czbiohub-mosquito/contigs/SAMPLE/contigs.fasta`

To setup blast db, follow https://czbiohub.atlassian.net/wiki/spaces/DS/pages/903905690/nt+nr+BLAST+etc+on+EC2


1. Download blast db


3. To setup BLAST, we follow the instructions in 